In [8]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

In [6]:
df = pd.read_csv("C:\\Users\\admin\\OneDrive - BOMBAY COATED AND SPECIAL STEELS PRIVATE LIMITED\\Desktop\\customer_support_ticket_analyzer\\data\\customer_support_tickets_1.csv")
df.head()

,message_id,user_name,location,email_subject,message_body,created_at,true_category
0,M0001,Ryan Obrien,North Billy,Law from traditional now Mrs.,Reflect available century join outside. i cant...,2025-09-01 06:10:59,Feature Request
1,M0002,Jamie Salazar,South Kari,Require billion probably cut raise include now.,Try cause behind single project. Sport sound c...,2025-06-16 04:58:19,Other
2,M0003,Clinton Wallace,Port Paige,South maintain year firm.,While travel major strong pull. Us history lig...,2025-08-17 14:54:02,Feature Request
3,M0004,Christopher Olsen,Jeanville,Fill personal fire.,Can nothing force move free body stand approac...,2025-10-15 06:27:28,Praise
4,M0005,Alex Alexander,West Gail,Term authority offer feeling than.,Control skin fall. Left worker ready take prop...,2025-05-14 22:35:51,Praise


Load and inspect the dataset (CSV format). Identify	missing	values,	duplicates, and	column types.	

In [ ]:
df.isna().sum() # Check for missing values

message_id       0
user_name        0
location         0
email_subject    0
message_body     0
created_at       0
true_category    0
dtype: int64

In [17]:
df.columns.value_counts() # Check for duplicate columns

message_id       1
user_name        1
location         1
email_subject    1
message_body     1
created_at       1
true_category    1
Name: count, dtype: int64

In [20]:
type(df.columns) # Check column types

pandas.core.indexes.base.Index